1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음.
        - 문서 길이(input)가 길어 답변 생성에 시간 소요
3. 임베딩(수치화)
    - 벡터 데이터베이스에 저장
4. 질문이 있으면 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
# 0. 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [51]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고 분할 #######################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 백터 데이터베이스에 저장  #########################

## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

## 2.3 벡터 데이터베이스에 저장(in memory)
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
)

# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  #############

## 3.1 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###########

## 4.1 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)
 
## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI()

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message.content

"전세사기피해자를 위한 금융지원은 '전세사기피해자 지원 및 주거안정에 관한 특별법'에 따라 제도화되어 있습니다. 이 법은 전세사기로 피해를 입은 임차인에게 경ㆍ공매 절차 및 조세 징수 등에 관한 특례를 부여하여 전세사기피해자를 지원하고 주거안정을 도모하는 것을 목적으로 합니다.\n\n전세사기피해자는 법에 정의된 요건을 충족한 임차인으로, 국토교통부장관이 결정한 자를 말합니다. 이 법에 따라 전세사기피해지원위원회가 심의하여 결정된 전세사기피해자들은 금융지원을 받을 수 있습니다. 이를 통해 피해를 입은 전세사기피해자들이 재정적인 어려움에서 벗어나고 주거 안정을 유지할 수 있도록 도와줍니다.\n\n따라서, 전세사기피해자들이 임대차계약으로 발생한 피해를 보상받을 수 있도록 금융지원이 제공되고 있습니다."